In [ ]:
!pip install flask_ngrok
!pip install pyngrok==4.1.1
!ngrok authtoken '2TioxnD798loTkSsBpLfO4qxXye_7oNWrdUPco9E8x5Easuke'

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=094ab1965d2d840db1aabea0c453b81237fff8b98c3ec018cba5c75f64071b31
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
import pickle
import math
import string
import socket
import os
from urllib.parse import urlparse
import requests

from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/random_forest_classifier.pkl', 'rb') as file:
    clf = pickle.load(file)

In [ ]:

def get_domain_length(url):
    """
    Returns the length of the entire URL.
    """
    return len(url)


def get_domain_entropy(url):
    """
    Returns the entropy of the domain name.
    """
    domain = urlparse(url).netloc
    alphabet = string.ascii_lowercase + string.digits
    freq = [0] * len(alphabet)
    for char in domain:
        if char in alphabet:
            freq[alphabet.index(char)] += 1
    entropy = 0
    for count in freq:
        if count > 0:
            freq_ratio = float(count) / len(domain)
            entropy -= freq_ratio * math.log(freq_ratio, 2)
    return round(entropy,2)


def is_ip_address(url):
    """
    Returns True if the URL uses an IP address instead of a domain name.
    """
    domain = urlparse(url).netloc
    try:
        socket.inet_aton(domain)
        return 1
    except socket.error:
        return 0


def has_malicious_extension(url):
    _, ext = os.path.splitext(url)
    malicious_extensions = ['.exe', '.dll', '.bat', '.cmd', '.scr', '.js', '.vbs',
                            '.hta', '.ps1', '.jar', '.py', '.rb']

    if ext.lower() in malicious_extensions:
        return 1
    else:
        return 0


def query_params_count(url):
    """
    Returns Number of query parameters in the URL
    """
    parsed = urlparse(url)
    query_params = parsed.query.split('&')
    if query_params[0] == '':
        return 0
    else:
        return len(query_params)


def path_tokens_count(url):
    """
    Returns Number of path tokens
    """
    parsed = urlparse(url)
    path_tokens = parsed.path.split('/')
    path_tokens = [token for token in path_tokens if token]  # remove empty tokens
    return len(path_tokens)


def hyphens_count(url):
    """
    Returns the number of hyphens in the entire URL
    """
    parsed = urlparse(url)
    return url.count('-')


def digits_count(url):
    """
    Returns Number of digits in the entire URL
    """
    return sum(c.isdigit() for c in url)


def has_special_characters(url):
    special_chars = ['@', '!', '#', '$', '%', '^', '&', '*', '_', '+']
    for char in special_chars:
        if char in url:
            return 1
    return 0


def getInputArray(url):
    result = []
    result.append(get_domain_length(url))
    result.append(get_domain_entropy(url))
    result.append(is_ip_address(url))
    result.append(has_malicious_extension(url))
    result.append(query_params_count(url))
    result.append(path_tokens_count(url))
    result.append(hyphens_count(url))
    result.append(digits_count(url))
    result.append(has_special_characters(url))
    return result

In [ ]:
def predict_urls(urls):
    features = [getInputArray(url) for url in urls]
    predictions = clf.predict(features)
    return predictions

In [ ]:
import requests
def check_Nortan_WebSafe(url):
    response = requests.get(
        f"https://safeweb.norton.com/report/show?url={url}")
    html_content = response.text
    if "known dangerous webpage" in html_content:
        return False
    else:
        return True

In [ ]:
check_Nortan_WebSafe("https://acconut.acesvpass.card.noboliving.com")

False

In [ ]:
@app.route('/check_url', methods=['POST'])
def check_url():
    data = request.get_json()
    url = data['url']
    norton_result = int(check_Nortan_WebSafe(url))
    prediction_result = int(predict_urls([url])[0])

    final_result = norton_result if norton_result else (prediction_result and norton_result)

    return jsonify({'norton_result': norton_result, 'prediction_result': prediction_result, 'final_result': final_result})

In [10]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://5e14-34-150-222-5.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [05/Nov/2023 06:31:29] "POST /check_url HTTP/1.1" 200 -
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [05/Nov/2023 06:38:51] "POST /check_url HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Nov/2023 06:38:59] "GET /check_url HTTP/1.1" 405 -
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [05/Nov/2023 06:40:39] "POST /check_url HTTP/1.1" 200 -
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names,